# USRP UCLouvain: Tutorial Guide 

Ce notebook a pour objectif de guider les utilisateurs dans la configuration et l'utilisation d'un setup expérimental basé sur des USRPs (Universal Software Radio Peripheral), disponibles à l'UCLouvain. Ce matériel est largement utilisé dans les domaines de la recherche et de l'enseignement en télécommunications et en traitement du signal pour la radio logicielle, offrant une flexibilité exceptionnelle pour l'implémentation et l'expérimentation de diverses techniques de modulation, de transmission et de réception de signaux.

Le notebook est structuré en plusieurs sections :

* Matériel disponible : Cette section décrit en détail le matériel mis à disposition, notamment les modèles d'USRPs, les antennes et les accessoires nécessaires.
* Connexion aux USRPs : Vous serez guidé à travers les étapes de connexion aux USRPs, depuis la configuration réseau jusqu'à la vérification de la connectivité avec l'ordinateur de contrôle.
* Transmission d'un signal OFDM : Un exemple détaillé est fourni pour illustrer la génération et la transmission d'un signal OFDM (Orthogonal Frequency Division Multiplexing). Vous apprendrez à configurer les paramètres du signal, à le transmettre à l'aide des USRPs et à le recevoir.
* Traitement Com et Radar à la réception : La dernière section présente des techniques de traitement du signal pour des applications de communication (Com) et radar (Radar). Vous apprendrez à analyser les signaux reçus, à estimer les paramètres de canal et à extraire des informations pertinentes pour des applications radar ainsi qu'estimer les symboles de données transmis.

Ce notebook est conçu pour faciliter l'apprentissage et la mise en pratique des concepts clés en traitement du signal et radio logicielle, en offrant une approche pratique et interactive à travers l'utilisation de matériel réel.

Document écrit par:
* Jérôme Lafontaine
* François De Saint Moulin
* Martin Willame

(Note de l'auteur: Introduction ChatGPTed)

## 0. Infos générales

### 0.1. Matériel à disposition du setup
Le setup de base consiste en:
- 2 USRPs NI2944R (équivalent chez Etus: X310) (1 Tx (Bluey) et un Rx (Bingo)) 
- 1 écran (connetion écran: TX via Displayport et RX via DVI)
- 1 octoclock

Note:
- Post-its sur tours pour TX et RX, TX = Bluey, RX = Bingo
- Si un seul écran: Pour changer d'ordinateur, il faut modifier les connexions USB du clavier et de la souris. De plus, en appuyant sur le bouton "+" et "-" de l'écran, il est possible de sélectionner une entrée graphique différente.
- Allumage en connectant la prise murale + bouton d'allumage des ordinateurs et des usrps (pas de bouton sur l'octoclock). Pour éteindre tout le setup, débrancher la prise murale après avoir éteint les USRPs avec le bouton et avoir éteint proprement les ordinateurs

### 0.2. Accès aux ordinateurs
- Session Assistants:<br>
Nom de compte: usrp_config <br>
Mot de passe: usrpadmin <br>

- Session Students: 
Pour les étudiants, il existe une session usrp_user(s)? -> il faut tester pour voir si cela marche sans les droits admins


### 0.3 Configuration de la session
<!> Les étapes définies dans les chapitres 1 (Configuration des cartes ethernet) et 2 (Exécution) doivent être exécutée à chaque allumage des PCs, et sur les deux PCs ! <!>



## 1. Configuration des cartes ethernet (Tx et Rx) 

Utiliser le fichier "ethsetup.sh" sur le bureau pour configurer les différentes cartes ethernet et la connexion avec les USRPs. Voici quelques explications pour en comprendre le contenu :

### 1.1. Explications

--> les USRPs ont tous un nom et une adresse IP inscrite au crayon sur les boîtiers !

--> Une adresse IP a la forme xxx.xxx.xxx.xxx/yy. yy définit le nombre de bits définissant le masque de sous-réseau. xxx représentent les numéros de 8 bits sous forme décimales, de 1 à 255

ex: 192.168.50.1/24 -> 11000000.10101000.00110010.00000001 en forme binaire

24 => 24 premier bits définissent le sous-réseau, c'est-à-dire les trois premiers chiffres. 192.168.50.zz est le sous-réseau encodé sur 24 bits, et zz définira l'addresse du PC, de l'USRP, ... (dans cet exemple c'est 1).

--> le fichier ethsetup.sh configure les différentes cartes réseaux du PC, pour que ces dernières

    - soient actives

    - soient connectés aux sous-réseaux voulus 

    - communiquent avec un MTU (maximum transmission unit) de 9000 bytes/packet (limite demandée par les USRPs)

    - Pour vérifier si on détecte des usrps

In [ ]:
uhd_find_devices

A Savoir: <br> 
interface_name = nom de chaque port ethernet sur chaque carte. Par example: 
* enp1s0f0 = premier port de la première carte, <br>
* enp1s0f1 = deuxième port de la première carte, <br>
* enp2s0f1 = deuxième port de la deuxième carte <br>
    
Chaque PC a normalement deux cartes réseaux, chacune ayant deux interfaces (= connexions ethernet) 

<!> Une carte par PC n'est pas branchée pour le moment (19/09/24). Par ailleurs, ces cartes ne sont pas assurées de pouvoir fonctionner à pleine vitesse, car le second port PCIe du PC n'est pas suffisamment grand <!>

De plus, chaque PC a une interface supplémentaire pour la carte réseau connectée à internet, du style enp0s31f6

### 1.2. Exécution

Toutes ces commandes sont comprises dans un fichier appelé ethsetup.sh qu'il faut exécuter:

a. Ouvrir le terminal depuis le bureau (clic droit sur le bureau -> open terminal)

b. Executer la commande 

Normalement, plusieurs messages devraient s'afficher, notamment s'il y a des erreurs.

Si la commande ne peut pas s'executer (erreur "command not found"), c'est que les permissions ne sont pas accordées pour l'exécution du fichier. Pour régler ce problème, il faut accorder la permission d'execution avec la commande suivante "chmod +rx nomdufichier"

Une fois qu'il n'y a plus d'erreurs (certaines erreur peuvent aparaitre pour certains noms d'interface qui ne sont pas connecté, ceci n'est donc pas un problème), la configuration des cartes ethernet s'affiche, par exemple

Voici à quoi corresponde les informations que l'on peut lire:

* Interface = enp1s0f0 <br>
* UP = l'interface est actif <br>
* RUNNING = l'interface est connecté <br>
* MTU 9000 => l'interface va communiquer à la bonne vitesse <br>
* inet = addresse IP de l'interface <br>
* netmask = masque de sous-réseau (équivalent au /yy, mais sous-forme xxx.xxx.xxx.xxx). Ici, 255.255.255.0 signifie que les 24 premiers bits forment le masque de sous-réseau (équivalent à /24). Pour l'addresse 192.168.40.1, le sous-réseau est 192.168.40.yy.

A la fin, le fichier "ethsetup.sh" vérifie s'il détecte un USRP. Si c'est le cas, quelque chose de ce style est affiché :

Ici, l'USRP jerome ayant l'adresse 192.168.40.11 est bien connecté au réseau.

<!> Si le setup est utilisé dans un but pédagogique, il faudrait que l'assistant du cours suive les étapes jusqu'ici, puis se déconnecte pour se reconnecter sur la session users (qui n'ont pas accès à sudo), à vérifier si cela marche ! <!>

## 2. Création du disque RAM (Tx et Rx)

Le but est de créer un espace sur le bureau (appelé disque) permettant de copier-coller des fichiers de code source directement dans la RAM. Cela permettra alors au code de directement lire/écrire les données dans la RAM:

<!> Cette étape est a réaliser même si le dossier RAMDisk est déjà présent sur le bureau car il n'est pas associé à un espace dans la RAM par défaut. Si le dossier n'existe pas, il faut d'ailleurs le créer <!>

size = taille allouée de notre disque RAM (ici, 8.192 Mo). Le PC contient 32 Go de RAM, mais tout ne peut pas être alloué sinon le PC va dysfonctionner ! <br>
/export/home/usrpconfig/Desktop/RAMDisk = chemin d'accès vers le bureau. Pour créer un disque RAM sur la session étudiante, il faut changer usrpconfig en usrpusers !

--> On devrait maintenant observer un dossier appelé "RAMDisk" sur le bureau.

## 3. Transmission d'un signal (Tx uniquement)

<!> Toutes les étapes de cette section s'effectuent sur le PC Bluey ! La génération des différents fichiers nécessaires est décrites par après. <!>

Pour effectuer la transmission, deux fichiers doivent être copiés dans notre disque RAM:

a. le fichier "signal.txt" - c'est le fichier contenant le signal à transmettre. Les lignes contiennent successivement les valeurs I et Q de chaque échantillon complexe du signal à transmettre. Un exemple de génération de ce code est fournis pour une transmission OFDM dans la section 5 ("Génération d'un signal à transmettre (Tx uniquement)") de ce document. <br>
b. le fichier "tx_waveforms_radar" - c'est le fichier C++ compilé qui va effectuer la transmission. La façon de générer ce fichier ainsi que son utilité est discutée dans la section 7 ("Compilation des codes C++") de ce document. Ce fichier se trouve dans le dossier "./Desktop/C++ Files/build-dir".

Ensuite il faut ouvrir le terminal depuis le disque RAM (clic droit dans le dossier -> ouvrir terminal), et exécuter cette commande:

Si la commande ne peut pas s'executer (erreur "command not found"), c'est que les permissions ne sont pas accordées pour l'exécution du fichier. Pour régler ce problème, il faut accorder la permission d'execution avec la commande suivante "chmod +rx nomdufichier". Si "permission denied", il faut sudo.

Voici une description des différents argumments de la commande:
* name = nom de l'USRP, inscrit sur le boitier. Si cela ne fonctionne pas, l'USRP peut aussi être référé via son adresse IP en écrivant <addr=xxx.xxx.xxx.xxx> à la place de <name=sam> <br>
* subdev = définition des antennes et des cartes filles de l'USRP à utiliser pour la transmission (voir https://files.ettus.com/manual/page_configuration.html). A priori il ne faut pas modifier cela, excepté pour utiliser les deux cartes filles en transmission ("A:0 B:0"). <br>
* ant = quelle antenne est utilisée sur la carte fille <br> 
* rate = fréquence d'échantillonage (ici 200 MHz) <br>
* freq = fréquence porteuse (ici 3.2 GHz) <br>
* sig_len = longueur du signal en échantillons complexe (exemple 6491520, du coup le fichier signal.txt contient 12983040 lignes) <br>
* spb = nombre d'échantillons dans un buffer. Ce nombre doit être un entier multiple de *sig_len*. <br>
* file = nom du fichier contenant le signal à transmettre <br>
* gain = gain de l'antenne de transmission (ici 30 dB) <br>
* ref = référence pour l'horloge (ici external, c'est-à-dire qu'on utilise une horloge externe)

<!> <to_complete> est à compléter à chaque fois que l'on utilise un nouveau signal selon la valeur fournie par "usrp_tx.py" présenté dans la section 5 (Génération du signal à transmettre) <!>

<!> les USRPs n'acceptent pas toutes les valeurs possibles pour les configurations. Par exemple, il n'est pas possible de choisir un rate autre que 100 et 200 MHz dans cette plage de valeur. Pour vérifier les valeurs possibles, il faut chipoter avec l'USRP (il existe des fonctions C++/Python pour récupérer les plages possibles). Malgré tout, il est possible de vérifier si la configuration voulue a fonctionné car l'USRP affiche les valeurs configurées (voir juste après les lignes "Setting ... / Actual ...") <!>

Voici un exemple d'affichage suite à l'exécution de la commande :

Les nombreux L qui se sont affichés apparaissent car l'USRP essaie de transmettre quelque chose qu'il n'a pas encore réussi à lire. Le PC prend un certain temps pour remplir son propre buffer, et l'USRP essaie de le lire trop tôt. Malgré tout, ceux-ci n'ont aucun impact sur la transmission, si la réception ne s'exécute pas directement.

A partir de ce moment-là, le signal est transmis continuellement en boucle, jusqu'à ce que l'utilisateur stoppe la transmission avec CTRL + C.

## 4. Réception d'un signal (Rx uniquement)

<!> Toutes les étapes de cette section s'effectuent sur le PC Bingo ! La génération des différents fichiers nécessaires est décrites par après. <!>

Pour effectuer la transmission, un seul fichier doit être copié dans notre disque RAM: le fichier "adv_rx_waveforms_radar" - c'est le fichier C++ compilé qui va effectuer la réception. Ce fichier se trouve dans le dossier "./Desktop/C++ Files/build-dir".

Ensuite il faut ouvrir le terminal depuis le disque RAM (clic droit dans le dossier -> ouvrir terminal), et exécuter cette commande:

Si la commande ne peut pas s'executer (erreur "command not found"), c'est que les permissions ne sont pas accordées pour l'exécution du fichier. Pour régler ce problème, il faut accorder la permission d'execution avec la commande suivante "chmod +rx nomdufichier". Si "permission denied", il faut sudo.

Voici une description des différents argumments de la commande:

* name = nom de l'USRP, inscrit sur le boitier. Si cela ne fonctionne pas, l'USRP peut aussi être référé via son adresse IP en écrivant <addr=xxx.xxx.xxx.xxx> à la place de <name=jerome> <br>
* subdev = définition des antennes et des cartes filles de l'USRP à utiliser pour la transmission (voir https://files.ettus.com/manual/page_configuration.html). A priori il ne faut pas modifier cela, excepté pour utiliser les deux cartes filles en réception ("A:0 B:0"). <br>
* ant = quelle antenne est utilisée sur la carte fille <br> 
* sync et ref = méthode de synchronisation et référence pour l'horloge (ici external, c'est-à-dire qu'on utilise une horloge externe) <br>
* rate = fréquence d'échantillonage (ici 200 MHz) <br>
* freq = fréquence porteuse (ici 3.2 GHz) <br>
* gain = gain de l'antenne de réception (ici 30 dB). Il peut y avoir un warning si les antennes sont trop proches <br>
* nsamps = nombre d'échantillons à sauvegarder, égal à 2 fois le nombre d'échantillons transmis pour la synchronisation <br>
* filename = nom du fichier où sauvegarder le signal reçu <br>
* rx_bw = bande passante du filtre de réception <br>
* channels = canal à enregistrer, à ne pas modifier lors de l'utilisation d'une unique antenna <br>
* nbr_meas = nombre de mesure successive réalisée de taille nsamps <br>
* secs = délai en secondes entre deux mesures successives


<!> les USRPs n'acceptent pas toutes les valeurs possibles pour les configurations. Par exemple, il n'est pas possible de choisir un rate autre que 100 et 200 MHz dans cette plage de valeur. Pour vérifier les valeurs possibles, il faut chipoter avec l'USRP (il existe des fonctions C++/Python pour récupérer les plages possibles). Malgré tout, il est possible de vérifier si la configuration voulue a fonctionné car l'USRP affiche les valeurs configurées (voir juste après les lignes "Setting ... / Actual ...") <!>

Voici un exemple d'affichage suite à l'exécution de la commande :

Généralement, lorsque le signal est bien reçu, la valeur maximum I/Q avoisine les 0.25 (alors qu'elle descend à 0.005 si c'est une mesure à vide). S'il y a interférence d'un autre système, par exemple le Wi-Fi à 2.4 GHz, cette valeur peut monter jusqu'à 0.9.

## 5. Génération d'un signal à transmettre (Tx uniquement)

Le code suivant décrit la procédure pour la transmission d'un signal OFDM. Il est repris ici à titre d'exemple puisque tout utilisateur peut modifier le code pour transmettre toute forme d'onde (simple ton, signal FMCW, ...). La fonction s'appelle "usrp_tx.py" et est complètement commentée.

<!> 
Le code fournis utilise la librairie "radcomlib" développée par François De Saint Moulin à l'UCLouvain 

La librairie radcomlib est installée sur les ordinateurs du setup et peut être utilisée comme telle. Si vous souhaitez l'installer sur votre propre ordinateur via Anaconda, voici les étapes:
1) Télécharger l'archive "RADCOM_library" et décompresser le dossier

2) Ouvrir l'anaconda prompt en mode admin depuis le dossier "RADCOM_library"

3) Executer la commande "python setup.py install" ou "python setup.py develop" (la seconde option permet de modifier les fonctions du package alors que la première ne tient pas compte des modifications dans les fichiers de la librairie faites par l'utilisateur)

<!> 

Une fois le package installé ou déjà disponible, le code pour la génération du signal à transmettre est le suivant:

## 6. Traitement du signal reçu

Le code suivant décrit la procédure pour la récepetion d'un signal OFDM afin de réaliser un traitement Com et Radar. Il est repris ici à titre d'exemple puisque tout utilisateur peut modifier le code pour transmettre toute forme d'onde (simple ton, signal FMCW, ...). La fonction s'appelle "usrp_rx.py" et est complètement commentée.

<!> Le code fournis utilise la librairie "radcomlib" développée par François De Saint Moulin à l'UCLouvain, celle-ci est disponible sur les ordinateurs du setup expérimental <!> 

Le code est le suivant:

## 7. Compilation des codes C++

Si on souhaite faire une modif dans les fichiers C++ (attention, uniquement si grosse modification du setup car c'est compliqué de travailler avec le code C originel), il est possible de modifier puis de recompiler les fichiers executable. Pour ce faire, voici les différentes étapes:

1) Copier-coller le dossier C++ Files (il ne sagirait pas de perdre les executables originel)

2) Dans ce nouveau dossier, supprimer les éxecutables précédents

3) ouvrir un terminal dans le dossier (il faut que cmake soit installé (vérif avec "cmake --version") (c'est le cas sur le pc Tx, à vérifier sur le pc Rx)

4) commande: "cmake ." => execute cmake avec le fichier se trouvant dans le dossier

5) commande: "make" => les fichiers executable sont construits et peuvent être utilisés dans RAMDisk

## 8. Tips and Tricks

* A la réception, on enregistre toujours au moins 2 fois la taille de ce que l'on a transmis. En effet, l'émetteur transmet en continu le signal demandé. Pour le récuperer, on écoute au moins 2 fois la taille du signal transmis afin de s'assurer d'avoir au moins une copie complète du signal. Pour la corrélation avec le préambule (servant à la détection du début du signal), on ne corrèle qu'avec la moitié du signal reçu. On est alors certain d'avoir au moins un préambule complet dans cette partie et de pouvoir récupérer un symbol complet de donnée après le préambule. Une fois le début du préambule trouvé, on jette tout ce qui se trouve avant et on garde uniquement le préambule et les données qui viennent après. 

* Parler de la polarisation des antennes

* Utilisation des FPGAs pour les Usrps (voir les liens "tips" sur le pc Tx de l'ordinateur expérimental)

* Ordre de grandeur des tailles de fichier qui fonctionne pour la transmission: toujours au moins 100 000, entre 1 et 3 millions ça marche bien. Le fichier ne peut pas être trop court et ne peut pas être trop long.